In [21]:
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LSTM
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import numpy as np
import pandas as pd
from collections import deque
import random
import time
import os

In [2]:
SEQ_LEN = 60
future_pre = 3
ratio_to_pre = "LTC-USD"

epochs = 10
b_size = 64
NAME = f"{SEQ_LEN}-SEQ-{future_pre}-pre-{int(time.time())}"

def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

main_df = pd.DataFrame()

ratios = ["BTD-USD", "LTC-USD", "ETH-USD", "BCH-USD"]
for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    
    df = pd.read_csv("crypto_data/LTC-USD.csv", names=["time", "low", "high", "open", "close", "volume"])
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_vloume"}, inplace=True)
    
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_vloume"]]
    
    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
        
        
main_df['future'] = main_df[f"{ratio_to_pre}_close"].shift(-future_pre)
main_df['target'] = list(map(classify, main_df[f"{ratio_to_pre}_close"], main_df["future"]))

In [3]:
#preprocess
def preprocess_df(df):
    df = df.drop('future', 1)

    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change()
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values)
            
    df.dropna(inplace=True)
    
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN)
    for i in df.values:
        prev_days.append([n for n in i[:-1]])
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])

    #balance
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
            
    random.shuffle(buys)
    random.shuffle(sells)

    lower = min(len(buys), len(sells))
    
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys+sells
    
    random.shuffle(sequential_data)
    
    x = []
    y = []
    
    for seq, target in sequential_data:
        x.append(seq)
        y.append(target)
        
    return np.array(x), np.array(y)

In [4]:
#create normalized and ready-to-train/test data
times = sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

In [28]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.4)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

model = Sequential()

model.add(CuDNNLSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(CuDNNLSTM(128, return_sequences=True))
model.add(Dropout(0.2)) #0.1
model.add(BatchNormalization())

model.add(CuDNNLSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2, activation="softmax"))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#tensorboard = TensorBoard(log_dir=f"logs\\{NAME}")

#filepath = "RNN_Final.model"
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

#filepath = "RNN_First.model"  # unique file name that will include the epoch and the validation acc for that epoch
#checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')  # saves only the best ones

#checkpoint_dir="./models"
#callbacks = [ModelCheckpoint(filepath=checkpoint_dir + "/crpyto_1", verbose=1, save_best_only=True, mode='max'), tensorboard]

checkpoint = ModelCheckpoint('models\\{epoch:02d}.model', verbose=1, save_freq=2)

model.fit(train_x, train_y, batch_size=b_size, epochs=epochs, validation_data=(validation_x, validation_y), callbacks=[checkpoint])

Train on 81642 samples, validate on 4018 samples
Epoch 1/10

Epoch 00001: saving model to models\01.model
    0/81642 [..............................] - ETA: 0s

ValueError: Layer has 2 states but was passed 0 initial states.

In [30]:
%reload_ext tensorboard

In [31]:
%tensorboard --logdir logs/ #http://localhost:6006/

Reusing TensorBoard on port 6006 (pid 1624), started 4:38:14 ago. (Use '!kill 1624' to kill it.)